In [1]:
# os, to see if a file exists
import os
# tabular data manipulation
import numpy as np
import pandas as pd
# data visualizations
import matplotlib.pyplot as plt
import seaborn as sns
# train test split
from sklearn.model_selection import train_test_split
# access to sql query
import wrangle as w

Acquire dataset via wrangle.py file

In [2]:
df = w.new_zillow_data()

Review the data

    - First Impressions
    - Look for Null or Missing Values

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 6 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   fips                          float64
dtypes: float64(6)
memory usage: 98.6 MB


In [4]:
df.shape

(2152863, 6)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.0,3.00,3.0,4.0,25.0
bathroomcnt,2152852.0,2.230688,0.999280,0.0,2.00,2.0,3.0,32.0
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.0,1257.00,1623.0,2208.0,952576.0
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.0,188170.25,327671.0,534527.0,98428909.0
yearbuilt,2143526.0,1960.949681,22.162196,1801.0,1949.00,1958.0,1976.0,2016.0
fips,2152863.0,6048.377335,20.433292,6037.0,6037.00,6037.0,6059.0,6111.0


In [6]:
df.head(10)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips
0,0.0,0.0,NaN,27516.0,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6037.0
5,0.0,0.0,NaN,124.0,NaN,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,6037.0
8,0.0,0.0,NaN,6730242.0,NaN,6037.0
9,0.0,0.0,NaN,15532.0,NaN,6037.0


    - Are the floats neccessary? 
    - How many observations are missing square feet?
    - Is year built worth retaining?
    - Rename columns for simplicity

In [12]:
df = df.rename(columns={'bedroomcnt': 'beds', 
                   'bathroomcnt': 'baths', 
                   'calculatedfinishedsquarefeet' : 'sqft', 
                   'taxvaluedollarcnt': 'tax', 
                   'yearbuilt': 'year', 
                   'fips': 'county'})

In [13]:
df.isna().sum()

beds        11
baths       11
sqft      8484
tax        493
year      9337
county       0
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   beds    float64
 1   baths   float64
 2   sqft    float64
 3   tax     float64
 4   year    float64
 5   county  float64
dtypes: float64(6)
memory usage: 98.6 MB


    - 215000 + observations
    - 8500 ~ square footage data missing
    - 500 ~ tax value data missing
    - 9300 + year built data missing
    - all types are float
    - are the observations with missing data all in the same rows?

Roughly 4% of the dataset's observations contain null values

In [10]:
round(9300/215000, 2)*100

4.0

    - Boolean Masking to evaluate whether the null values are in the same rows
    - How many rows?

In [15]:
df['sqft'].isna().value_counts()

False    2144379
True        8484
Name: sqft, dtype: int64

In [16]:
df['year'].isna().value_counts()

False    2143526
True        9337
Name: year, dtype: int64

    - 7877 rows with values that are null.
    - 3.65% of total dataset
    - The impact from dropping these rows will be negligable

In [23]:
df[(df['sqft'].isna()) & (df['year'].isna())]

,beds,baths,sqft,tax,year,county
0,0.0,0.0,NaN,27516.0,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,6037.0
5,0.0,0.0,NaN,124.0,NaN,6037.0
...,...,...,...,...,...,...
2152823,0.0,0.0,NaN,2568893.0,NaN,6059.0
2152844,0.0,0.0,NaN,92679.0,NaN,6111.0
2152854,0.0,0.0,NaN,1198476.0,NaN,6037.0
2152857,0.0,0.0,NaN,1087111.0,NaN,6059.0
